# Integração e Bibliotecas

In [ ]:
# Passo 0 - Entender o desafio que você quer resolver.

In [ ]:
#Espaço Reservado para importação de bibliotecas
import pandas as pd  # Para Installar o pandas > pip install pandas
import matplotlib.pyplot as plt  
import plotly.express as px 
import seaborn as sns
import smtplib
import email.message as em 
from babel.numbers import format_currency


In [ ]:
# Passo 1 - Importar a base de dados (Pasta Vendas)

vendas = pd.read_excel('dataSets/vendas.xlsx')
display(vendas) 

In [ ]:
display(vendas) 

In [ ]:
# Passo 2 - Calcular o produto mais vendido (em quantidade)

#Agrupamos pro produto e somamos todas as colunas numericas
tb_vendas_produto_quantidade = vendas.groupby('Produto').sum(numeric_only=True)

#Filtramos da tabela apenas a Colunas Index (Produto) e a Coluna Quantidade
tb_vendas_produto_quantidade = tb_vendas_produto_quantidade[['Quantidade']]

#Ordenamos nossa tabela de decrescente pela coluna Quantidade
tb_vendas_produto_quantidade = tb_vendas_produto_quantidade.sort_values(by='Quantidade', ascending=False)

tb_vendas_produto_quantidade.to_excel('tb_vendas_produto.xlsx','Vendas_Qtd_Prod')



display(tb_vendas_produto_quantidade)

In [ ]:
# Passo 3 - Calcular o produto mais vendido (em faturamento)
#Primeiro criamos a coluna Faturamento 
vendas['Faturamento'] = vendas['Quantidade'] * vendas['Valor Unitário']

#Agrupamos pro produto e somamos todas as colunas numericas
tb_vendas_produto_faturamento = vendas.groupby('Produto').sum(numeric_only=True)

#Filtramos da tabela apenas a Colunas Index (Produto) e a Coluna Quantidade
tb_vendas_produto_faturamento = tb_vendas_produto_faturamento[['Faturamento']]

#Ordenamos nossa tabela de decrescente pela coluna Quantidade
tb_vendas_produto_faturamento = tb_vendas_produto_faturamento.sort_values(by='Faturamento', ascending=False)

#Cria lambda para formatação de moeda 
lb_formata_real = lambda v: format_currency(v, 'BRL', locale='pt_BR')

#Uso a lamba para formatar minha nova tabela
tb_vendas_produto_faturamento_f = pd.DataFrame(tb_vendas_produto_faturamento['Faturamento'].apply(lb_formata_real).reset_index())



display(tb_vendas_produto_faturamento_f)


In [ ]:
# Passo 4 - Calcular a loja/estado que mais vendeu (em faturamento)
#Agrupando dados por Loja
tb_loja_faturamento = vendas.groupby('Loja').sum(numeric_only=True)

#Filtramos as colunas deixando apenas Loja (Index)
tb_loja_faturamento = tb_loja_faturamento[['Faturamento']].sort_values(by='Faturamento', ascending=False)

tb_loja_faturamento


In [ ]:
# Passo 5 - Calcular o ticket médio por loja/estadom. (Somar Valor de cada item e divide pelo total de vendas)
#Criar a coluna Ticket Medio
vendas['Ticket Médio'] = vendas['Valor Unitário']

#Agrupar por loja e fazer a media de todos os demais 
tb_ticket_medio = vendas.groupby('Loja').mean(numeric_only=True)

#Remove as demais colunas deixando apenas 'Ticket Médio' e Loja (Index)
tb_ticket_medio = tb_ticket_medio[['Ticket Médio']].sort_values(by='Ticket Médio', ascending=False)

tb_ticket_medio

In [ ]:
# Passo 6 - Calcular a Quantidade de produtos Vendidos por Loja

#Agrupando por produto e somando tudo
tb_produto_vendido_loja = vendas.groupby('Loja').sum(numeric_only=True)

#filtrando as colunas a serem exibidas
tb_produto_vendido_loja = tb_produto_vendido_loja[['Quantidade', 'Faturamento']]

#Ordena os dados
tb_produto_vendido_loja = tb_produto_vendido_loja.sort_values(by='Quantidade', ascending=False)

tb_produto_vendido_loja


In [ ]:
# Passo 7 - Criar um gráfico/dashboard da loja/estado que mais vendeu (em faturamento) <Resultado do Passo 4>

#Criando um grafico de barras com ploty
grafico_px = px.bar(tb_loja_faturamento, y='Faturamento', x=tb_loja_faturamento.index)
grafico_px.show()


#Criando Grafico com Matplot

plt.bar(tb_loja_faturamento.index, tb_loja_faturamento['Faturamento'])

plt.title('Faturamento por Lojas')
plt.xlabel('Loja')
plt.ylabel('Faturamento em M')

plt.xticks(rotation=90)
plt.show()

""""
plt.figure(12,8)

sns.barplot(x=tb_loja_faturamento.index, y='Faturamento', data=tb_loja_faturamento, palette='viridis')

plt.title('Faturamento por Lojas')
plt.xlabel('Loja')
plt.ylabel('Faturamento em M')

plt.tight_layout()

plt.show()

"""

In [ ]:
# Passo 8 - Enviar um e-mail para o setor responsável.  (Enviar Item 2, 3, 4 )

 
corpo_email = f"""
<p>Prezados, tudo bem?</p>
<p>Segue Analise de Vendas</p>
<p> <b>Produto Mais Vendido Faturamento</b> {tb_vendas_produto_faturamento_f.to_html(index=False, justify='center')} </p>

"""

msg = em.Message()
msg['Subject'] = "Teste em Python" # Assunto do email

msg['From'] = 'rotinabackup58@gmail.com' #Conta de Email que irá enviar   
password = 'fceveunimwskylxg'    #Senha do email que enviará (Senha de app criada) 

msg['To'] = 'servico.backup.status@gmail.com'  #Email destino, quem vai receber   

msg.add_header('Content-Type', 'text/html')

msg.set_payload(corpo_email)

#Faz login do Servidor do gmail e envia o email.
s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()  #Ativa Criptografia TLS
s.login(msg['From'], password)
s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

print('Email enviado')